# EduHub E-Learning Platform  

## Part 1: Database Setup & Data Modeling


This portfolio submission showcases the first step in building EduHub’s backend: designing and provisioning the MongoDB schema that will power user management, course content and learning analytics.

**Objectives**  

- **Connect** to our MongoDB instance (`eduhub_db`) via `pymongo`.  
- **Enforce** strict JSON Schema validation on six collections:
  - `users` – tracks students and instructors  
  - `courses` – course metadata, including `rating` and `ratingsCount` for later analytics  
  - `enrollments` – which students are in which courses  
  - `lessons` – individual lesson content and ordering  
  - `assignments` – tasks with due dates  
  - `submissions` – student work, grades and feedback  
- **Generate** one representative sample document per collection.  
- **Display** each sample as a clean, interactive table using pandas in Jupyter.

**Implementation Highlights**  
1. **Database Connection**  
   Established via `get_client()`, verifying connectivity with a server info check.  
2. **Schema Validation**  
   Used `"validationLevel": "strict"` and `"validationAction": "error"` to guarantee that only valid documents can be inserted.  
3. **Sample Data**  
   Created example documents (including `rating` fields on courses) and wrapped them in pandas DataFrames.  
4. **Visualization**  
   Leveraged pandas’ HTML rendering for polished tables directly in the notebook.


In [1]:
from IPython.display import display, Markdown
import part1_setup as part1_setup

# Provision & fetch
client = part1_setup.get_client()
part1_setup.setup_database(client)
samples = part1_setup.get_sample_docs()

# Render as markdown tables
for name, df in samples.items():
    title = f"### {name.capitalize()} Sample\n"
    md_table = df.to_markdown(index=False)
    display(Markdown(title + md_table))


[OK] Collections created with JSON Schema validation.


### Users Sample
| userId   | email             | firstName   | lastName   | role    | dateJoined                 | profile                                                                   | isActive   |
|:---------|:------------------|:------------|:-----------|:--------|:---------------------------|:--------------------------------------------------------------------------|:-----------|
| u1001    | alice@example.com | Alice       | Anderson   | student | 2025-06-17 04:06:33.496372 | {'bio': 'Data enthusiast', 'avatar': '', 'skills': ['Python', 'MongoDB']} | True       |

### Courses Sample
| courseId   | title                     | description                                     | instructorId   | category         | level    |   duration |   price | tags               |   rating |   ratingsCount | createdAt                  | updatedAt                  | isPublished   |
|:-----------|:--------------------------|:------------------------------------------------|:---------------|:-----------------|:---------|-----------:|--------:|:-------------------|---------:|---------------:|:---------------------------|:---------------------------|:--------------|
| c2001      | Intro to Data Engineering | Learn the basics of MongoDB and data pipelines. | u2001          | Data Engineering | beginner |        8.5 |   99.99 | ['MongoDB', 'ETL'] |        0 |              0 | 2025-06-17 04:06:33.496372 | 2025-06-17 04:06:33.496372 | False         |

### Enrollments Sample
| enrollmentId   | userId   | courseId   | enrolledAt                 |   progress | completionStatus   |
|:---------------|:---------|:-----------|:---------------------------|-----------:|:-------------------|
| e3001          | u1001    | c2001      | 2025-06-17 04:06:33.496372 |          0 | in progress        |

### Lessons Sample
| lessonId   | courseId   | title                        | content                          |   order | createdAt                  | updatedAt                  |
|:-----------|:-----------|:-----------------------------|:---------------------------------|--------:|:---------------------------|:---------------------------|
| l4001      | c2001      | Getting Started with MongoDB | Installation and first CRUD ops. |       1 | 2025-06-17 04:06:33.496372 | 2025-06-17 04:06:33.496372 |

### Assignments Sample
| assignmentId   | courseId   | title               | description                     | dueDate                    | createdAt                  | updatedAt                  |
|:---------------|:-----------|:--------------------|:--------------------------------|:---------------------------|:---------------------------|:---------------------------|
| a5001          | c2001      | Setup Your Database | Install and connect to MongoDB. | 2025-06-24 04:06:33.496372 | 2025-06-17 04:06:33.496372 | 2025-06-17 04:06:33.496372 |

### Submissions Sample
| submissionId   | assignmentId   | userId   | submittedAt                |   grade | feedback   |
|:---------------|:---------------|:---------|:---------------------------|--------:|:-----------|
| s6001          | a5001          | u1001    | 2025-06-18 04:06:33.496372 |       0 |            |

# Part 2: Data Population


Programmatically inserted realistic sample data into each of our six collections, ensuring that every foreign-key reference points to a valid parent document.


1. **Cleared** all existing documents in `users`, `courses`, `enrollments`, `lessons`, `assignments`, and `submissions`.  
2. **Generated & Inserted**:  
   - 20 users (5 instructors, 15 students)  
   - 8 courses across distinct categories  
   - 15 enrollments linking students ↔ courses  
   - 25 lessons spread across those courses  
   - 10 assignments tied to courses  
   - 12 submissions tied to assignments & students  
3. **Maintained Referential Integrity**:  
   - `instructorId` in **courses** → `users.userId`  
   - `userId` & `courseId` in **enrollments** → `users.userId`, `courses.courseId`  
   - `courseId` in **lessons**, **assignments** → `courses.courseId`  
   - `assignmentId` & `userId` in **submissions** → `assignments.assignmentId`, `users.userId`  


In [2]:
# configure notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# import and run
import part2_data_population as part2_data_population
tables = part2_data_population.main()

# pandas will render each DataFrame
tables["users"]
tables["courses"]
tables["enrollments"]
tables["lessons"]
tables["assignments"]
tables["submissions"]



[OK] Sample data inserted.


,submissionId,assignmentId,userId,submittedAt,grade,feedback,_id
0,s6001,a5008,u1002,2024-06-24 04:06:33.533217,28,Needs improvement.,6850e9c924ef14dd39507d87
1,s6002,a5002,u1006,2024-08-31 04:06:33.533217,75,Good job!,6850e9c924ef14dd39507d88
2,s6003,a5009,u1004,2024-07-27 04:06:33.533217,28,Needs improvement.,6850e9c924ef14dd39507d89
3,s6004,a5001,u1002,2025-02-15 04:06:33.533217,29,Needs improvement.,6850e9c924ef14dd39507d8a
4,s6005,a5002,u1015,2024-08-31 04:06:33.533217,42,Needs improvement.,6850e9c924ef14dd39507d8b
5,s6006,a5002,u1009,2024-09-03 04:06:33.533217,35,Needs improvement.,6850e9c924ef14dd39507d8c
6,s6007,a5008,u1004,2024-06-21 04:06:33.533217,92,Good job!,6850e9c924ef14dd39507d8d
7,s6008,a5010,u1010,2024-11-20 04:06:33.533217,31,Needs improvement.,6850e9c924ef14dd39507d8e
8,s6009,a5008,u1013,2024-06-30 04:06:33.533217,24,Needs improvement.,6850e9c924ef14dd39507d8f
9,s6010,a5002,u1002,2024-09-06 04:06:33.533217,45,Needs improvement.,6850e9c924ef14dd39507d90


## Part 2.2: Demonstrating Referential Integrity

Below we merge key collections to prove that every foreign-key reference points to a valid parent document:

1. **Courses → Instructors**  
2. **Enrollments → Students & Courses**  


In [3]:
from IPython.display import Markdown, display

# Assume `tables` has been populated by part2_data_population.main()
users_df       = tables["users"]
courses_df     = tables["courses"]
enrollments_df = tables["enrollments"]

# 1) Courses with their instructor’s name
instructors_df = users_df[users_df.role == "instructor"][["userId", "firstName", "lastName"]]
courses_with_instructor = courses_df.merge(
    instructors_df,
    left_on="instructorId",
    right_on="userId"
)[["courseId", "title", "firstName", "lastName"]]

display(Markdown(
    "### Courses → Instructors\n" +
    courses_with_instructor.to_markdown(index=False)
))

# 2) Enrollments with student name and course title
students_df = users_df[users_df.role == "student"][["userId", "firstName", "lastName"]]
enrollments_with_details = (
    enrollments_df
    .merge(students_df, on="userId")
    .merge(courses_df[["courseId", "title"]], on="courseId")
    [["enrollmentId", "firstName", "lastName", "title", "progress", "completionStatus"]]
)

display(Markdown(
    "### Enrollments → Students & Courses\n" +
    enrollments_with_details.to_markdown(index=False)
))


### Courses → Instructors
| courseId   | title                         | firstName   | lastName   |
|:-----------|:------------------------------|:------------|:-----------|
| c3001      | Data Science Essentials       | Instructor4 | Smith      |
| c3002      | Web Development Essentials    | Instructor4 | Smith      |
| c3003      | Machine Learning Essentials   | Instructor3 | Smith      |
| c3004      | Cloud Computing Essentials    | Instructor1 | Smith      |
| c3005      | UI/UX Design Essentials       | Instructor1 | Smith      |
| c3006      | Cybersecurity Essentials      | Instructor3 | Smith      |
| c3007      | Mobile Development Essentials | Instructor2 | Smith      |
| c3008      | Databases Essentials          | Instructor5 | Smith      |

### Enrollments → Students & Courses
| enrollmentId   | firstName   | lastName   | title                         |   progress | completionStatus   |
|:---------------|:------------|:-----------|:------------------------------|-----------:|:-------------------|
| e4001          | Student4    | Jones      | Data Science Essentials       |         51 | in progress        |
| e4002          | Student5    | Jones      | Web Development Essentials    |         72 | in progress        |
| e4003          | Student15   | Jones      | Cybersecurity Essentials      |         83 | in progress        |
| e4004          | Student8    | Jones      | Mobile Development Essentials |         58 | in progress        |
| e4005          | Student3    | Jones      | UI/UX Design Essentials       |         31 | in progress        |
| e4006          | Student12   | Jones      | UI/UX Design Essentials       |         54 | in progress        |
| e4007          | Student15   | Jones      | Mobile Development Essentials |         28 | in progress        |
| e4008          | Student3    | Jones      | Databases Essentials          |         96 | in progress        |
| e4009          | Student1    | Jones      | Web Development Essentials    |         80 | in progress        |
| e4010          | Student3    | Jones      | Mobile Development Essentials |          8 | in progress        |
| e4011          | Student7    | Jones      | Mobile Development Essentials |         59 | in progress        |
| e4012          | Student9    | Jones      | UI/UX Design Essentials       |          1 | in progress        |
| e4013          | Student11   | Jones      | Web Development Essentials    |         68 | in progress        |
| e4014          | Student13   | Jones      | UI/UX Design Essentials       |         43 | in progress        |
| e4015          | Student2    | Jones      | UI/UX Design Essentials       |         20 | in progress        |

In [4]:
# In a single cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import part3_crud as part3_crud
client = part3_crud.get_client()
db = client[part1_setup.DB_NAME]

# Call any function you like, e.g.:
part3_crud.get_active_students(db)
part3_crud.get_course_details(db, "c9999")


,_id,userId,email,firstName,lastName,role,dateJoined,profile,isActive
0,6850e9c924ef14dd39507d3e,u1001,student1@example.com,Student1,Jones,student,2025-02-23 04:06:33.533,"{'bio': '', 'avatar': '', 'skills': []}",True
1,6850e9c924ef14dd39507d3f,u1002,student2@example.com,Student2,Jones,student,2025-04-26 04:06:33.533,"{'bio': '', 'avatar': '', 'skills': []}",True
2,6850e9c924ef14dd39507d41,u1004,student4@example.com,Student4,Jones,student,2025-06-01 04:06:33.533,"{'bio': '', 'avatar': '', 'skills': []}",True
3,6850e9c924ef14dd39507d42,u1005,student5@example.com,Student5,Jones,student,2025-05-01 04:06:33.533,"{'bio': '', 'avatar': '', 'skills': []}",True
4,6850e9c924ef14dd39507d43,u1006,student6@example.com,Student6,Jones,student,2025-02-18 04:06:33.533,"{'bio': '', 'avatar': '', 'skills': []}",True
5,6850e9c924ef14dd39507d44,u1007,student7@example.com,Student7,Jones,student,2024-09-03 04:06:33.533,"{'bio': '', 'avatar': '', 'skills': []}",True
6,6850e9c924ef14dd39507d47,u1010,student10@example.com,Student10,Jones,student,2024-08-20 04:06:33.533,"{'bio': '', 'avatar': '', 'skills': []}",True
7,6850e9c924ef14dd39507d48,u1011,student11@example.com,Student11,Jones,student,2025-06-14 04:06:33.533,"{'bio': '', 'avatar': '', 'skills': []}",True
8,6850e9c924ef14dd39507d4a,u1013,student13@example.com,Student13,Jones,student,2024-12-25 04:06:33.533,"{'bio': '', 'avatar': '', 'skills': []}",True
9,6850e9c924ef14dd39507d4b,u1014,student14@example.com,Student14,Jones,student,2025-03-30 04:06:33.533,"{'bio': '', 'avatar': '', 'skills': []}",True


TypeError: 'NoneType' object is not subscriptable

# Part 3: Basic CRUD Operations

This section demonstrate end-to-end Create, Read, Update and Delete operations on our MongoDB collections using PyMongo. All operations are wrapped in functions in the `part3_crud` module and return pandas DataFrames, which are render as clean Markdown tables below.

---

## Tasks Covered

1. **Create Operations**  
   - Add a new student user  
   - Create a new course  
   - Enroll a student in a course  
   - Add a new lesson to a course  

2. **Read Operations**  
   - Find all active students  
   - Retrieve course details (with instructor info)  
   - Get all courses in a specific category  
   - List students enrolled in a particular course  
   - Search courses by title (case-insensitive, partial match)  

3. **Update Operations**  
   - Update a user’s profile information  
   - Mark a course as published  
   - Update assignment grades  
   - Add tags to an existing course  

4. **Delete Operations**  
   - Soft-delete a user (set `isActive = false`)  
   - Delete an enrollment  
   - Remove a lesson from a course  

---

## Implementation Highlights

- **Modular functions** in `part3_crud.py` encapsulate each operation.  
- **pandas DataFrames** returned by each function allow immediate tabular display.  
- **Pure-Markdown tables** (`df.to_markdown(index=False)`) ensure a polished, console-friendly look.  
- All operations are executed in a single notebook cell for an **end-to-end** demonstration.


In [5]:


# 1) Ensure we only see what we explicitly display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "none"

# 2) Helpers for Markdown rendering
from IPython.display import Markdown, display

# 3) (Re)populate Part 2 data so we start from a known state
import part2_data_population as part2_data_population
tables2 = part2_data_population.main()

# 4) Import setup and CRUD modules & get DB handle
import part1_setup as part1_setup, part3_crud as part3_crud
client = part1_setup.get_client()
db = client[part1_setup.DB_NAME]

# 5) Perform all CRUD tasks and collect their DataFrames
results = {}

# --- Task 3.1: CREATE ---
results["Created Student (u9001)"] = part3_crud.create_student(
    db, userId="u9001", email="jane.doe@example.com",
    firstName="Jane", lastName="Doe"
)
results["Created Course (c9001)"] = part3_crud.create_course(
    db, courseId="c9001", title="Automated Testing 101",
    instructorId="u2001", category="Software Testing",
    level="beginner", duration=4.0, price=49.99,
    tags=["testing", "automation"]
)
results["Enrolled Student (e9001)"] = part3_crud.enroll_student(
    db, enrollmentId="e9001", userId="u9001", courseId="c9001"
)
results["Added Lesson (l9001)"] = part3_crud.add_lesson(
    db, lessonId="l9001", courseId="c9001",
    title="Introduction to Test Automation",
    content="Overview of tools and frameworks.", order=1
)

# --- Task 3.2: READ ---
results["Active Students"] = part3_crud.get_active_students(db)
results["Course Details (c9001)"] = part3_crud.get_course_details(db, "c9001")
results["Courses in 'Software Testing'"] = part3_crud.get_courses_by_category(db, "Software Testing")
results["Students in Course c9001"] = part3_crud.get_students_in_course(db, "c9001")
results["Search Courses 'Testing'"] = part3_crud.search_courses_by_title(db, "Testing")

# --- Task 3.3: UPDATE ---
results["Updated Profile (u9001)"] = part3_crud.update_user_profile(
    db, userId="u9001",
    profile_updates={"bio": "QA engineer in training", "skills": ["Selenium", "pytest"]}
)
results["Published Course (c9001)"] = part3_crud.publish_course(db, "c9001")
results["Updated Submission Grade (s6001)"] = part3_crud.update_submission_grade(db, "s6001", 95)
results["Added Tags to Course (c9001)"] = part3_crud.add_course_tags(
    db, courseId="c9001", tags=["ci/cd", "pytest"]
)

# --- Task 3.4: DELETE ---
results["Soft-deleted User (u9001)"] = part3_crud.soft_delete_user(db, "u9001")
results["Removed Enrollment (e9001)"] = part3_crud.remove_enrollment(db, "e9001")
results["Deleted Lesson (l9001)"] = part3_crud.delete_lesson(db, "l9001")

# 6) Render each result as a Markdown table
for title, df in results.items():
    display(Markdown(f"## {title}\n\n{df.to_markdown(index=False)}\n"))


[OK] Sample data inserted.


## Created Student (u9001)

| userId   | email                | firstName   | lastName   | role    | dateJoined                 | profile                                 | isActive   | _id                      |
|:---------|:---------------------|:------------|:-----------|:--------|:---------------------------|:----------------------------------------|:-----------|:-------------------------|
| u9001    | jane.doe@example.com | Jane        | Doe        | student | 2025-06-17 04:06:50.446735 | {'bio': '', 'avatar': '', 'skills': []} | True       | 6850e9da24ef14dd39507df0 |


## Created Course (c9001)

| courseId   | title                 | description   | instructorId   | category         | level    |   duration |   price | tags                      |   rating |   ratingsCount | createdAt                  | updatedAt                  | isPublished   | _id                      |
|:-----------|:----------------------|:--------------|:---------------|:-----------------|:---------|-----------:|--------:|:--------------------------|---------:|---------------:|:---------------------------|:---------------------------|:--------------|:-------------------------|
| c9001      | Automated Testing 101 |               | u2001          | Software Testing | beginner |          4 |   49.99 | ['testing', 'automation'] |        0 |              0 | 2025-06-17 04:06:50.447785 | 2025-06-17 04:06:50.447785 | False         | 6850e9da24ef14dd39507df1 |


## Enrolled Student (e9001)

| enrollmentId   | userId   | courseId   | enrolledAt                 |   progress | completionStatus   | _id                      |
|:---------------|:---------|:-----------|:---------------------------|-----------:|:-------------------|:-------------------------|
| e9001          | u9001    | c9001      | 2025-06-17 04:06:50.449264 |          0 | in progress        | 6850e9da24ef14dd39507df2 |


## Added Lesson (l9001)

| lessonId   | courseId   | title                           | content                           |   order | createdAt                  | updatedAt                  | _id                      |
|:-----------|:-----------|:--------------------------------|:----------------------------------|--------:|:---------------------------|:---------------------------|:-------------------------|
| l9001      | c9001      | Introduction to Test Automation | Overview of tools and frameworks. |       1 | 2025-06-17 04:06:50.449936 | 2025-06-17 04:06:50.449936 | 6850e9da24ef14dd39507df3 |


...

## Updated Submission Grade (s6001)

| _id                      | submissionId   | assignmentId   | userId   | submittedAt                |   grade | feedback   |
|:-------------------------|:---------------|:---------------|:---------|:---------------------------|--------:|:-----------|
| 6850e9da24ef14dd39507de3 | s6001          | a5010          | u1007    | 2025-06-03 04:06:50.428000 |      95 | Good job!  |


## Added Tags to Course (c9001)

| _id                      | courseId   | title                 | description   | instructorId   | category         | level    |   duration |   price | tags                                         |   rating |   ratingsCount | createdAt                  | updatedAt                  | isPublished   |
|:-------------------------|:-----------|:----------------------|:--------------|:---------------|:-----------------|:---------|-----------:|--------:|:---------------------------------------------|---------:|---------------:|:---------------------------|:---------------------------|:--------------|
| 6850e9da24ef14dd39507df1 | c9001      | Automated Testing 101 |               | u2001          | Software Testing | beginner |          4 |   49.99 | ['testing', 'automation', 'ci/cd', 'pytest'] |        0 |              0 | 2025-06-17 04:06:50.447000 | 2025-06-17 04:06:50.461000 | True          |


## Soft-deleted User (u9001)

| _id                      | userId   | email                | firstName   | lastName   | role    | dateJoined                 | profile                                                                            | isActive   |
|:-------------------------|:---------|:---------------------|:------------|:-----------|:--------|:---------------------------|:-----------------------------------------------------------------------------------|:-----------|
| 6850e9da24ef14dd39507df0 | u9001    | jane.doe@example.com | Jane        | Doe        | student | 2025-06-17 04:06:50.446000 | {'bio': 'QA engineer in training', 'avatar': '', 'skills': ['Selenium', 'pytest']} | False      |


## Removed Enrollment (e9001)




## Deleted Lesson (l9001)




# Part 4: Advanced Queries & Aggregation


Advanced data retrieval and analytics tasks on our MongoDB collections using PyMongo. Demonstrates both stand-alone queries (Task 4.1) and multi-stage aggregation pipelines (Task 4.2) to surface business insights.

---

### Task 4.1: Complex Queries  
- **Price Filtering:** Find all courses with price between \$50 and \$200  
- **Recent Users:** Retrieve users who joined in the last 6 months  
- **Tag Search:** Find courses that include specified tags using the `$in` operator  
- **Due-Soon Assignments:** Get assignments with due dates within the next week  

### Task 4.2: Aggregation Pipelines  
1. **Course Enrollment Statistics**  
   - Total enrollments per course  
   - Average course rating per category  
2. **Course-Level Metrics**  
   - Enrollment counts per course  
   - Completion rate per course  
3. **Student Performance Analysis**  
   - Average grade per student  
   - Identification of top-performing students (≥ 75th percentile)  
4. **Instructor Analytics**  
   - Total distinct students taught by each instructor  
   - Average rating across their courses  
   - Total revenue generated per instructor  
5. **Advanced Analytics**  
   - Monthly enrollment trends over the past year  
   - Most popular course categories by enrollment  
   - Average student progress per course  

---

## Implementation Highlights

- **Modular Functions:** Each query/pipeline is encapsulated in a clear, reusable function in `part4_aggregation.py`.  
- **Pure PyMongo Pipelines:** Leverages lookups, grouping, projection, and conditional logic directly in MongoDB.  
- **Pandas DataFrames:** All results return as DataFrames, enabling seamless conversion to Markdown tables via `df.to_markdown()`.  
- **Single-Cell Demonstration:** A single notebook cell runs every query and renders the outputs as clean, self-documented Markdown tables.

Below you will see a sequence of Markdown-styled tables, one per query and aggregation, confirming that our data model supports the advanced analytical use cases outlined in the brief.  


In [6]:
# Configure notebook to show all DataFrames
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Run our updated Part 4 aggregation module
import part4_aggregation as part4_aggregation
tables = part4_aggregation.main()

# Render each DataFrame as a clean Markdown table
from IPython.display import Markdown, display
for name, df in tables.items():
    display(Markdown(f"### {name.replace('_', ' ').title()}\n\n{df.to_markdown(index=False)}\n"))



=== Courses By Price ===
+--------------------------+------------+-----------------------------+---------------------------------------------+----------------+------------------+--------------+------------+---------+------------------------------------+----------+----------------+----------------------------+----------------------------+---------------+
| _id                      | courseId   | title                       | description                                 | instructorId   | category         | level        |   duration |   price | tags                               |   rating |   ratingsCount | createdAt                  | updatedAt                  | isPublished   |
+==========================+============+=============================+=============================================+================+==================+==============+============+=========+====================================+==========+================+============================+===========================

### Courses By Price

| _id                      | courseId   | title                       | description                                 | instructorId   | category         | level        |   duration |   price | tags                               |   rating |   ratingsCount | createdAt                  | updatedAt                  | isPublished   |
|:-------------------------|:-----------|:----------------------------|:--------------------------------------------|:---------------|:-----------------|:-------------|-----------:|--------:|:-----------------------------------|---------:|---------------:|:---------------------------|:---------------------------|:--------------|
| 6850e9da24ef14dd39507da9 | c3001      | Data Science Essentials     | An introductory course on Data Science.     | u2005          | Data Science     | beginner     |        5.9 |  101.57 | ['Data Science', 'Data']           |        0 |              0 | 2024-10-03 04:06:50.428000 | 2024-10-03 04:06:50.428000 | True          |
| 6850e9da24ef14dd39507dab | c3003      | Machine Learning Essentials | An introductory course on Machine Learning. | u2002          | Machine Learning | advanced     |       13.9 |  123.88 | ['Machine Learning', 'Machine']    |        0 |              0 | 2024-08-30 04:06:50.428000 | 2024-08-30 04:06:50.428000 | False         |
| 6850e9da24ef14dd39507dac | c3004      | Cloud Computing Essentials  | An introductory course on Cloud Computing.  | u2005          | Cloud Computing  | advanced     |       12.8 |  186.94 | ['Cloud Computing', 'Cloud']       |        0 |              0 | 2024-07-16 04:06:50.428000 | 2024-07-16 04:06:50.428000 | True          |
| 6850e9da24ef14dd39507dae | c3006      | Cybersecurity Essentials    | An introductory course on Cybersecurity.    | u2003          | Cybersecurity    | beginner     |        5.1 |  124.23 | ['Cybersecurity', 'Cybersecurity'] |        0 |              0 | 2024-10-26 04:06:50.428000 | 2024-10-26 04:06:50.428000 | True          |
| 6850e9da24ef14dd39507db0 | c3008      | Databases Essentials        | An introductory course on Databases.        | u2002          | Databases        | intermediate |        9.3 |   87.64 | ['Databases', 'Databases']         |        0 |              0 | 2025-05-13 04:06:50.428000 | 2025-05-13 04:06:50.428000 | False         |


### Recent Users

| _id                      | userId   | email                   | firstName   | lastName   | role       | dateJoined                 | profile                                                                            | isActive   |
|:-------------------------|:---------|:------------------------|:------------|:-----------|:-----------|:---------------------------|:-----------------------------------------------------------------------------------|:-----------|
| 6850e9da24ef14dd39507d97 | u2003    | instructor3@example.com | Instructor3 | Smith      | instructor | 2025-03-29 04:06:50.428000 | {'bio': 'Expert in subject 3', 'avatar': '', 'skills': ['Teaching', 'Leadership']} | True       |
| 6850e9da24ef14dd39507d98 | u2004    | instructor4@example.com | Instructor4 | Smith      | instructor | 2025-04-23 04:06:50.428000 | {'bio': 'Expert in subject 4', 'avatar': '', 'skills': ['Teaching', 'Leadership']} | True       |
| 6850e9da24ef14dd39507d9b | u1002    | student2@example.com    | Student2    | Jones      | student    | 2025-05-01 04:06:50.428000 | {'bio': '', 'avatar': '', 'skills': []}                                            | True       |
| 6850e9da24ef14dd39507d9c | u1003    | student3@example.com    | Student3    | Jones      | student    | 2025-03-24 04:06:50.428000 | {'bio': '', 'avatar': '', 'skills': []}                                            | False      |
| 6850e9da24ef14dd39507d9e | u1005    | student5@example.com    | Student5    | Jones      | student    | 2025-02-28 04:06:50.428000 | {'bio': '', 'avatar': '', 'skills': []}                                            | False      |
| 6850e9da24ef14dd39507d9f | u1006    | student6@example.com    | Student6    | Jones      | student    | 2025-05-18 04:06:50.428000 | {'bio': '', 'avatar': '', 'skills': []}                                            | True       |
| 6850e9da24ef14dd39507da4 | u1011    | student11@example.com   | Student11   | Jones      | student    | 2025-03-30 04:06:50.428000 | {'bio': '', 'avatar': '', 'skills': []}                                            | True       |
| 6850e9da24ef14dd39507da5 | u1012    | student12@example.com   | Student12   | Jones      | student    | 2025-01-17 04:06:50.428000 | {'bio': '', 'avatar': '', 'skills': []}                                            | True       |
| 6850e9da24ef14dd39507da6 | u1013    | student13@example.com   | Student13   | Jones      | student    | 2025-05-19 04:06:50.428000 | {'bio': '', 'avatar': '', 'skills': []}                                            | True       |
| 6850e9da24ef14dd39507da7 | u1014    | student14@example.com   | Student14   | Jones      | student    | 2025-01-08 04:06:50.428000 | {'bio': '', 'avatar': '', 'skills': []}                                            | True       |
| 6850e9da24ef14dd39507da8 | u1015    | student15@example.com   | Student15   | Jones      | student    | 2025-05-23 04:06:50.428000 | {'bio': '', 'avatar': '', 'skills': []}                                            | False      |
| 6850e9da24ef14dd39507df0 | u9001    | jane.doe@example.com    | Jane        | Doe        | student    | 2025-06-17 04:06:50.446000 | {'bio': 'QA engineer in training', 'avatar': '', 'skills': ['Selenium', 'pytest']} | False      |


### Courses By Tags




### Assignments Due Soon




...

### Student Performance

|   avgGrade | userId   | topPerformer   |
|-----------:|:---------|:---------------|
|        8   | u1006    | False          |
|        0   | u1001    | False          |
|       54.5 | u1007    | False          |
|       85   | u1004    | True           |
|       67   | u1012    | False          |
|       46   | u1015    | False          |
|       31.5 | u1011    | False          |
|       84   | u1010    | True           |


### Instructor Analytics

|   avgCourseRating |   totalRevenue | instructorId   |   numStudents |
|------------------:|---------------:|:---------------|--------------:|
|                 0 |         335.4  | u2002          |             3 |
|                 0 |         425.28 | u2005          |             5 |
|                 0 |         451.23 | u2003          |             5 |


### Monthlyenrollments

|   year |   month |   enrollments |
|-------:|--------:|--------------:|
|   2024 |       7 |             2 |
|   2024 |       8 |             2 |
|   2024 |       9 |             3 |
|   2024 |      10 |             1 |
|   2025 |       3 |             1 |
|   2025 |       4 |             3 |
|   2025 |       5 |             3 |


### Popularcategories

|   enrollments | category           |
|--------------:|:-------------------|
|             3 | Cybersecurity      |
|             3 | UI/UX Design       |
|             3 | Data Science       |
|             2 | Web Development    |
|             2 | Machine Learning   |
|             1 | Mobile Development |
|             1 | Databases          |


### Avgprogress

|   avgProgress | courseId   | title                         |
|--------------:|:-----------|:------------------------------|
|       67.3333 | c3006      | Cybersecurity Essentials      |
|       18.5    | c3003      | Machine Learning Essentials   |
|       29.3333 | c3005      | UI/UX Design Essentials       |
|       90      | c3008      | Databases Essentials          |
|       53.3333 | c3001      | Data Science Essentials       |
|       26      | c3002      | Web Development Essentials    |
|        5      | c3007      | Mobile Development Essentials |


# Part 5: Indexing & Performance

In this section we optimize our MongoDB queries by:

- **Task 5.1: Index Creation**  
  - `users.email`  
  - `courses.title`, `courses.category`  
  - `assignments.dueDate`  
  - `enrollments.userId`, `enrollments.courseId`  

- **Task 5.2: Query Optimization**  
  - Analyze query performance using explain() method in PyMongo
  - Optimize at least 3 slow queries
  - Document the performance improvements using Python timing functions



In [9]:


# Configure notebook to display all outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import Markdown, display
import part5_performance as part5_performance

# Run profiling and retrieve summary DataFrame
summary_df = part5_performance.main()

# Render the summary as a clean Markdown table
display(Markdown(
    "### Indexing & Query Performance Summary\n\n" +
    summary_df.to_markdown(index=False)
))


[ERROR] Explain failed: Explain cannot explain itself., full error: {'ok': 0.0, 'errmsg': 'Explain cannot explain itself.', 'code': 20, 'codeName': 'IllegalOperation'}
[ERROR] Explain failed: Explain cannot explain itself., full error: {'ok': 0.0, 'errmsg': 'Explain cannot explain itself.', 'code': 20, 'codeName': 'IllegalOperation'}
[ERROR] Explain failed: Explain cannot explain itself., full error: {'ok': 0.0, 'errmsg': 'Explain cannot explain itself.', 'code': 20, 'codeName': 'IllegalOperation'}
[ERROR] Could not create index on users: Index already exists with a different name: uix_users_email, full error: {'ok': 0.0, 'errmsg': 'Index already exists with a different name: uix_users_email', 'code': 85, 'codeName': 'IndexOptionsConflict'}
[OK] Created index courses.idx_courses_title
[OK] Created index courses.idx_courses_category
[OK] Created index assignments.idx_assignments_dueDate
[OK] Created index enrollments.idx_enrollments_userId
[OK] Created index enrollments.idx_enrollments_

### Indexing & Query Performance Summary

| Query                | Index                   | Server Before (ms)   |   Client Before (ms) | Server After (ms)   |   Client After (ms) | Server Improvement (%)   |   Client Improvement (%) |
|:---------------------|:------------------------|:---------------------|---------------------:|:--------------------|--------------------:|:-------------------------|-------------------------:|
| Courses by Category  | idx_courses_category    |                      |                  1   |                     |                 0.5 |                          |                     50   |
| Enrollments by User  | idx_enrollments_userId  |                      |                  1.2 |                     |                 0.5 |                          |                     58.3 |
| Assignments Due Soon | idx_assignments_dueDate |                      |                  0.4 |                     |                 0.8 |                          |                   -100   |

# Part 6: Data Validation & Error Handling
 
In this final section verifys that our JSON Schema validators (from Part 1) and error-handling routines correctly enforce data integrity and catch invalid operations.

**Tasks Covered**  
- **Task 6.1: Schema Validation**  
  - Required fields  
  - Data type checks  
  - Enum restrictions  
  - Email format via regex  
- **Task 6.2: Error Handling**  
  - Duplicate key errors  
  - Missing required fields  
  - Invalid data types  
  - Invalid enum values  
  - Invalid email formats  

Below we run a series of tests that intentionally violate each rule. The table that follows shows, for each test, the exact error message produced by MongoDB and caught in our Python code—proving that only schema-compliant data can be inserted into the database.

### Validation & Error Handling Results

| Test                    | Result                                                                     |
|-------------------------|----------------------------------------------------------------------------|
| Duplicate Key           | DuplicateKeyError caught: E11000 duplicate key error collection: eduhub_db.users index: _id_ dup key: { _id: … } |
| Missing Required Field  | WriteError caught (missing field): Document failed validation               |
| Invalid Data Type       | WriteError caught (invalid type): Document failed validation                |
| Invalid Enum Value      | WriteError caught (invalid enum): Document failed validation                |
| Invalid Email Format    | WriteError caught (invalid email): Document failed validation               |


In [8]:
# Part 6: Data Validation & Error Handling

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


from IPython.display import Markdown, display
import pandas as pd
import part6_validation as part6_validation

#  Runs the tests and collect results
results = part6_validation.main()

#  Builds a DataFrame for display
df = pd.DataFrame(
    list(results.items()),
    columns=["Test", "Result"]
)

#  Renders as a Markdown-style table
display(Markdown(
    "### Validation & Error Handling Results\n\n" +
    df.to_markdown(index=False)
))


[OK] Collections created with JSON Schema validation.


### Validation & Error Handling Results

| Test                   | Result                                                                                                                                              |
|:-----------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------|
| Duplicate Key          | DuplicateKeyError caught: E11000 duplicate key error collection: eduhub_db.users index: _id_ dup key: { _id: ObjectId('6850e9ed24ef14dd39507df7') } |
| Missing Required Field | WriteError caught (missing field): Document failed validation                                                                                       |
| Invalid Data Type      | WriteError caught (invalid type): Document failed validation                                                                                        |
| Invalid Enum Value     | WriteError caught (invalid enum): Document failed validation                                                                                        |
| Invalid Email Format   | WriteError caught (invalid email): Document failed validation                                                                                       |